In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard

train = pd.read_csv("./dataSet/train.csv")
test_orj  = pd.read_csv("./dataSet/test.csv")

print("Training set has {0[0]} rows and {0[1]} columns".format(train.shape))
print("Test set has {0[0]} rows and {0[1]} columns".format(test_orj.shape))

Using TensorFlow backend.


Training set has 42000 rows and 785 columns
Test set has 28000 rows and 784 columns


In [ ]:


# Easy model start from here!!!!!!!!!!!!!!!



In [ ]:
# Declare data set, optimizers and callback function for easy model
# put labels into y_train variable
Y_train = train["label"]
# Drop 'label' column
X_train = train.drop("label",axis = 1) 

X_train = X_train / 255.0
test = test_orj.copy() / 255.0

# Label Encoding 
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
Y_train = to_categorical(Y_train, num_classes = 10)

# Split the train and the validation set for the fitting
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)
print("x_train shape",X_train.shape)
print("x_val shape",X_val.shape)
print("y_train shape",Y_train.shape)
print("y_test shape",Y_val.shape)

# Select optimizer
lr = 1e-4
sgd = keras.optimizers.SGD(lr=lr, momentum=0.9)
rms_prop = keras.optimizers.RMSprop(lr=lr)
# adam = tf.contrib.optimizer_v2.AdamOptimizer
# adam = keras.optimizers.adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08) 
# adamax = keras.optimizers.Adamax(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
# adadelta = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)

# Declare callback functions
# Reduce the learning rate when gradient dicrease
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_acc',
    patience=3,
    verbose=1,
    factor=0.5,
    min_lr=0.0001
)

# Build tensorboard
tensorboard = TensorBoard(
    log_dir='./Graph',
    histogram_freq=0,
    write_graph=True,
    write_images=True
)

# Saving model callback function
def checkpoint(sequential):
    checkpoint_path = './keras_model/' + ('sequential' if sequential else 'functional') + '2.ckpt'
    check = ModelCheckpoint(
        checkpoint_path,
        save_weights_only=True,
        verbose=1,
    )
    return check

x_train shape (37800, 784)
x_val shape (4200, 784)
y_train shape (37800, 10)
y_test shape (4200, 10)


In [ ]:
# Keras sequential model
from keras.optimizers import RMSprop

batch_size = 128
num_classes = 10
epochs = 20

model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

# load model
model.load_weights('./keras_model/sequential1.ckpt')

model.compile(loss='categorical_crossentropy',
    optimizer=adam,
    metrics=['accuracy'])

history = model.fit(
    X_train, Y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_val, Y_val),
    callbacks=[learning_rate_reduction, tensorboard, checkpoint(True)],)
score = model.evaluate(X_val, Y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

result = model.predict(test, verbose=1)

# score = model.evaluate(X_val, Y_val, verbose=0)
# print(score)

In [ ]:
# Save result to csv

# result = model.predict(test, verbose=1)
result = np.argmax(result, axis=1)

image_id = []
for i in range(len(result)):
    image_id.append(i + 1)
d = {
    'ImageId': image_id,
    'Label': result,
}
result = pd.DataFrame(d)
result.to_csv('./result', index=False)

print('Save result to csv')

In [ ]:
# Keras funtional model
from keras.layers import Input, Dense
from keras.models import Model

batch_size = 128
num_classes = 10
epochs = 25

inputs = Input(shape=(784,))

x = Dense(512, activation='relu')(inputs)
x = Dense(512, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

model.compile(loss='categorical_crossentropy',
    optimizer=rms_prop,
    metrics=['accuracy'])

history = model.fit(
    X_train, Y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_val, Y_val),
    callbacks=[learning_rate_reduction, tensorboard, checkpoint(False)])
score = model.evaluate(X_val, Y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model1 = model
model.summary()


In [ ]:
# Tensorboard validation and train graph
import tensorflow as tf

writer_1 = tf.summary.FileWriter("./4logs/training")
writer_2 = tf.summary.FileWriter("./4logs/validation")

log_var = tf.Variable(0.0)
tf.summary.scalar("loss", log_var)
write_loss = tf.summary.merge_all()

session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())

i = 0
for train, validate in zip(history.history['loss'], history.history['val_loss']):
    summary = session.run(write_loss, {log_var: train})
    writer_1.add_summary(summary, i)
    writer_1.flush()

    summary = session.run(write_loss, {log_var: validate})
    writer_2.add_summary(summary, i)
    writer_2.flush()
    i += 1